In [1]:
# 1. Introduction to Kaggle

In [1]:
# --- Kaggle ---
import pandas as pd

test = pd.read_csv("KaggleData/test.csv")
test_shape = test.shape
print(test_shape)

train = pd.read_csv("KaggleData/train.csv")
train_shape = train.shape
print(train_shape)

(418, 11)
(891, 12)


In [2]:
# --- Torch ---
import numpy as np

test_t  = np.genfromtxt('KaggleData/kaggle_test.csv', delimiter=",")
print(test_t.shape)

train_t = np.genfromtxt('KaggleData/kaggle_train.csv', delimiter=",")
print(train_t.shape)

(419, 12)
(892, 13)


In [3]:
# 2. Exploring the data

In [4]:
# --- Kaggle ---
train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [5]:
# --- Torch ---
train_t[:2]

array([[  nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan],
       [ 1.  ,  0.  ,  3.  ,   nan,   nan,   nan, 22.  ,  1.  ,  0.  ,
          nan,  7.25,   nan,   nan]])

In [6]:
# --- Kaggle ---
import matplotlib.pyplot as plt

sex_pivot = train.pivot_table(index="Sex", values="Survived")
sex_pivot

,Survived
Sex,
female,0.742038
male,0.188908


In [7]:
# --- Torch ---
pandaDf = pd.DataFrame(train_t)
sex_pivot_t = pd.pivot_table(pandaDf, index="Sex", values="Survived")
sex_pivot_t

KeyError: 'Survived'

In [ ]:
# 3. Exploring and Converting the age column